In [1]:
#  Set up

#  install required packages
%pip install -q langgraph langchain langchain-openai python-dotenv

Note: you may need to restart the kernel to use updated packages.



[notice] A new release of pip is available: 25.2 -> 25.3
[notice] To update, run: C:\Users\hp\AppData\Local\Microsoft\WindowsApps\PythonSoftwareFoundation.Python.3.11_qbz5n2kfra8p0\python.exe -m pip install --upgrade pip


In [2]:
# Imports
from langgraph.graph import START, END, StateGraph, MessagesState
from langgraph.checkpoint.memory import MemorySaver
from langgraph.prebuilt import ToolNode, create_react_agent
from langchain_core.messages import HumanMessage, AIMessage, SystemMessage, ToolMessage
from langchain_core.tools import tool
from langchain_openai import ChatOpenAI
from dotenv import load_dotenv
from IPython.display import Image, display
from typing import Literal, TypedDict, Annotated
import operator
import os

print("✅ All imports successful")

✅ All imports successful


In [5]:
# Load API key
load_dotenv()
openai_api_key = os.getenv("OPENAI_API_KEY")

if not openai_api_key:
    raise ValueError("OPENAI_API_KEY not found!")

print("✅ API key loaded")

✅ API key loaded


In [6]:
# Initialize LLM
llm = ChatOpenAI(
    model="gpt-4o-mini",
    temperature=0,
    api_key=openai_api_key
)

print(f"✅ LLM initialized: {llm.model_name}")

✅ LLM initialized: gpt-4o-mini


In [11]:
# Custom state for Reflection
class ReflectionState(TypedDict):
    """State for reflection pattern."""
    clarity: str  # Original task
    completeness: str  # Current draft
    accuracy: str  # Critique of draft
    iterations: int  # Number of refinements
    final_output: str  # Final result

MAX_REFLECTIONS = 5  # Prevent infinite loops

print("✅ Reflection state defined")

✅ Reflection state defined


In [7]:

# Define Pydantic model for quality scoring
from pydantic import BaseModel, Field

class QualityScore(BaseModel):
    """Structured quality metrics for draft evaluation."""
    clarity: int = Field(..., ge=1, le=5, description="How clear is the content? (1-5)")
    completeness: int = Field(..., ge=1, le=5, description="Is it comprehensive? (1-5)")
    accuracy: int = Field(..., ge=1, le=5, description="How accurate? (1-5)")
    reasoning: str = Field(..., description="Explanation of scores")
    needs_refinement: bool = Field(..., description="Should we refine further?")

print("✅ QualityScore Pydantic model defined")


✅ QualityScore Pydantic model defined


In [ ]:

# System prompts for writer and critic

WRITER_PROMPT = """You are an expert writer and content creator.
Your task is to write a comprehensive, clear, and accurate response to the following task.

Task: {task}

{feedback}

Write a thoughtful and well-structured response."""

CRITIC_PROMPT = """You are a critical evaluator. Analyze the following draft and score it.

Original Task: {task}

Draft to Evaluate:
{draft}

Evaluate this draft on these criteria (1-5 scale):
1. **Clarity**: Is the writing clear and well-organized?
2. **Completeness**: Does it cover all important aspects?
3. **Accuracy**: Is the information correct and well-supported?

CRITICAL: Respond ONLY with valid JSON in this exact format (no other text):
{{
  "clarity": <number 1-5>,
  "completeness": <number 1-5>,
  "accuracy": <number 1-5>,
  "reasoning": "<string with explanation of scores and refinement feedback>",
  "needs_refinement": <true or false>
}}"""

print("✅ System prompts defined")


✅ System prompts defined


In [9]:

# Helper functions

def parse_quality_score(response_text: str) -> QualityScore:
    """Parse LLM response and extract structured QualityScore."""
    from langchain_core.output_parsers import JsonOutputParser
    
    parser = JsonOutputParser(pydantic_object=QualityScore)
    try:
        parsed = parser.parse(response_text)
        return QualityScore(**parsed)
    except Exception as e:
        print(f"Parse error: {e}")
        # Fallback: extract numbers from text
        import re
        clarity = int(re.search(r"clarity[:\s]*(\d)", response_text, re.I).group(1)) if re.search(r"clarity[:\s]*(\d)", response_text, re.I) else 3
        completeness = int(re.search(r"completeness[:\s]*(\d)", response_text, re.I).group(1)) if re.search(r"completeness[:\s]*(\d)", response_text, re.I) else 3
        accuracy = int(re.search(r"accuracy[:\s]*(\d)", response_text, re.I).group(1)) if re.search(r"accuracy[:\s]*(\d)", response_text, re.I) else 3
        return QualityScore(
            clarity=clarity,
            completeness=completeness,
            accuracy=accuracy,
            reasoning="Fallback scoring",
            needs_refinement=min(clarity, completeness, accuracy) < 4
        )

def format_score_display(iteration: int, score: QualityScore) -> str:
    """Format score for pretty display."""
    status = "🔄 REFINE" if score.needs_refinement else "✅ APPROVED"
    return f"\n📊 Iteration {iteration}: Clarity={score.clarity}, Completeness={score.completeness}, Accuracy={score.accuracy} → {status}"

print("✅ Helper functions defined")


✅ Helper functions defined


In [ ]:

# Adaptive Reflection Loop with Quality Metrics

def adaptive_reflection_workflow(task: str, max_iterations: int = MAX_REFLECTIONS) -> dict:
    """
    Run adaptive reflection with quality scoring.
    
    Args:
        task: The writing task/prompt
        max_iterations: Maximum refinement cycles
        
    Returns:
        Dictionary with final_output, scores history, and iteration count
    """
    
    draft = None
    scores_history = []
    feedback = ""
    
    print(f"\n{'='*60}")
    print(f"🎯 TASK: {task}")
    print(f"{'='*60}\n")
    
    for iteration in range(1, max_iterations + 1):
        # STEP 1: Write or refine draft
        print(f"📝 Iteration {iteration}: Writing/Refining...")
        
        writer_prompt = WRITER_PROMPT.format(
            task=task,
            feedback=f"Previous feedback to incorporate:\n{feedback}" if feedback else ""
        )
        
        draft_response = llm.invoke([HumanMessage(content=writer_prompt)])
        draft = draft_response.content
        
        # STEP 2: Get critic score
        print(f"🔍 Iteration {iteration}: Evaluating quality...")
        
        critic_prompt = CRITIC_PROMPT.format(
            task=task,
            draft=draft
        )
        
        # Add instruction for structured output
        critic_full_prompt = critic_prompt
        
        score_response = llm.invoke([HumanMessage(content=critic_full_prompt)])
        score_text = score_response.content
        
        # Parse score
        score = parse_quality_score(score_text)
        scores_history.append(score)
        
        # Display score
        print(format_score_display(iteration, score))
        print(f"   📋 Reasoning: {score.reasoning[:150]}...")
        
        # STEP 3: Check if approved
        if not score.needs_refinement:
            print(f"\n✅ APPROVED on iteration {iteration}!")
            print(f"\n🎉 Final Scores: Clarity={score.clarity}, Completeness={score.completeness}, Accuracy={score.accuracy}")
            break
        else:
            feedback = score.reasoning
            print(f"   → Will refine based on feedback...\n")
    
    # Return results
    return {
        "final_output": draft,
        "scores_history": scores_history,
        "iterations_completed": iteration,
        "final_score": score,
        "all_approved": not score.needs_refinement
    }

print("✅ Adaptive reflection workflow defined")


✅ Adaptive reflection workflow defined


In [13]:

# Run the adaptive reflection workflow

task = "Explain the importance of machine learning in modern software development in 2-3 paragraphs."

result = adaptive_reflection_workflow(task, max_iterations=5)

print("\n" + "="*60)
print("📊 FINAL RESULTS")
print("="*60)
print(f"\n✨ Final Output:\n{result['final_output']}\n")

# Display score improvements
print("\n📈 Score Progression:")
for i, score in enumerate(result['scores_history'], 1):
    print(f"  Iter {i}: Clarity={score.clarity}, Completeness={score.completeness}, Accuracy={score.accuracy}")

print(f"\n🏁 Total Iterations: {result['iterations_completed']}")
print(f"✅ All Approved: {result['all_approved']}")



🎯 TASK: Explain the importance of machine learning in modern software development in 2-3 paragraphs.

📝 Iteration 1: Writing/Refining...
🔍 Iteration 1: Evaluating quality...
🔍 Iteration 1: Evaluating quality...
Parse error: 1 validation error for QualityScore
reasoning
  Input should be a valid string [type=string_type, input_value={'clarity': 'The writing ... software development.'}, input_type=dict]
    For further information visit https://errors.pydantic.dev/2.12/v/string_type

📊 Iteration 1: Clarity=3, Completeness=3, Accuracy=3 → 🔄 REFINE
   📋 Reasoning: Fallback scoring...
   → Will refine based on feedback...

📝 Iteration 2: Writing/Refining...
Parse error: 1 validation error for QualityScore
reasoning
  Input should be a valid string [type=string_type, input_value={'clarity': 'The writing ... software development.'}, input_type=dict]
    For further information visit https://errors.pydantic.dev/2.12/v/string_type

📊 Iteration 1: Clarity=3, Completeness=3, Accuracy=3 → 🔄 REFIN

In [15]:
#another test Run the adaptive reflection workflow

task = "Explain stereochemistry and its importance in pharmaceutical compound in 2-3 paragraphs."

result = adaptive_reflection_workflow(task, max_iterations=5)

print("\n" + "="*60)
print("📊 FINAL RESULTS")
print("="*60)
print(f"\n✨ Final Output:\n{result['final_output']}\n")

# Display score improvements
print("\n📈 Score Progression:")
for i, score in enumerate(result['scores_history'], 1):
    print(f"  Iter {i}: Clarity={score.clarity}, Completeness={score.completeness}, Accuracy={score.accuracy}")

print(f"\n🏁 Total Iterations: {result['iterations_completed']}")
print(f"✅ All Approved: {result['all_approved']}")



🎯 TASK: Explain stereochemistry and its importance in pharmaceutical compound in 2-3 paragraphs.

📝 Iteration 1: Writing/Refining...
🔍 Iteration 1: Evaluating quality...
🔍 Iteration 1: Evaluating quality...
Parse error: 1 validation error for QualityScore
reasoning
  Input should be a valid string [type=string_type, input_value={'clarity': 'The writing ...own case in the field.'}, input_type=dict]
    For further information visit https://errors.pydantic.dev/2.12/v/string_type

📊 Iteration 1: Clarity=3, Completeness=3, Accuracy=3 → 🔄 REFINE
   📋 Reasoning: Fallback scoring...
   → Will refine based on feedback...

📝 Iteration 2: Writing/Refining...
Parse error: 1 validation error for QualityScore
reasoning
  Input should be a valid string [type=string_type, input_value={'clarity': 'The writing ...own case in the field.'}, input_type=dict]
    For further information visit https://errors.pydantic.dev/2.12/v/string_type

📊 Iteration 1: Clarity=3, Completeness=3, Accuracy=3 → 🔄 REFINE
  